In [ ]:
!pip install streamlit

In [5]:
# prompt: How to build a deployable streamlit web app that can gen (grayscale) imgs of handwritten digit of 0-9? The user should be able to specify which digit to generate, and each generation will gen 5 imgs. The output format should be the 28x28 MNIST dataset, and the model should be trained from scratch usin' Tensorflow w/ the MNIST dataset. Also print the model architecture and the loss function for ea trainin' iteration. Let's hv a 60-20-20 train-val-test split.

!pip install streamlit

import streamlit as st
import tensorflow as tf
import numpy as np
import random
import os

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Load and preprocess the MNIST dataset
(x_train_val, y_train_val), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values
x_train_val = x_train_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Expand dimensions to include channel dimension
x_train_val = np.expand_dims(x_train_val, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# One-hot encode the labels
y_train_val = tf.keras.utils.to_categorical(y_train_val, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Split training+validation data into training and validation sets (60-20-20 split)
total_samples = x_train_val.shape[0]
train_size = int(0.6 * total_samples)
val_size = int(0.2 * total_samples)

x_train, x_val = x_train_val[:train_size], x_train_val[train_size:train_size + val_size]
y_train, y_val = y_train_val[:train_size], y_train_val[train_size:train_size + val_size]

print(f"Training data shape: {x_train.shape}")
print(f"Validation data shape: {x_val.shape}")
print(f"Test data shape: {x_test.shape}")

# Build the Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(100 + 10,)))  # Latent noise (100) + Label (10)
    model.add(tf.keras.layers.Dense(7 * 7 * 128, use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Reshape((7, 7, 128)))
    assert model.output_shape == (None, 7, 7, 128)  # Note: None is the batch size

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

generator = build_generator()
generator.summary()

# Build the Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1)) # Output 1 for real, 0 for fake

    return model

discriminator = build_discriminator()
discriminator.summary()

# Define the GAN
noise_dim = 100
num_examples_to_generate = 5

@tf.function
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output, from_logits=True)
    fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output, from_logits=True)
    total_loss = real_loss + fake_loss
    return total_loss

@tf.function
def generator_loss(fake_output):
    return tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output, from_logits=True)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training step
@tf.function
def train_step(images, labels):
    noise = tf.random.normal([tf.shape(images)[0], noise_dim])
    # Concatenate noise with labels for conditional generation
    conditioned_noise = tf.concat([noise, tf.cast(labels, tf.float32)], axis=-1)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(conditioned_noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop (simplified for demonstration)
EPOCHS = 10
BATCH_SIZE = 64

# Create a dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(train_size).batch(BATCH_SIZE)

print("Starting training...")
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    total_gen_loss = 0.0
    total_disc_loss = 0.0
    num_batches = 0
    for image_batch, label_batch in train_dataset:
        gen_loss, disc_loss = train_step(image_batch, label_batch)
        total_gen_loss += tf.reduce_mean(gen_loss).numpy() # Convert to scalar
        total_disc_loss += tf.reduce_mean(disc_loss).numpy() # Convert to scalar
        num_batches += 1

    avg_gen_loss = total_gen_loss / num_batches
    avg_disc_loss = total_disc_loss / num_batches
    print(f"  Generator Loss: {avg_gen_loss:.4f}, Discriminator Loss: {avg_disc_loss:.4f}")

print("Training finished.")

# Streamlit App
def main():
    st.title("Handwritten Digit Generator")
    st.write("Generate grayscale images of handwritten digits using a trained GAN model.")

    digit_to_generate = st.selectbox("Select the digit to generate:", options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    if st.button("Generate Images"):
        st.write(f"Generating 5 images of digit {digit_to_generate}...")

        # Create a batch of noise and the specified label
        seed = tf.random.normal([num_examples_to_generate, noise_dim])
        # Create one-hot encoding for the specified digit
        labels = tf.one_hot([digit_to_generate] * num_examples_to_generate, depth=10)
        conditioned_seed = tf.concat([seed, tf.cast(labels, tf.float32)], axis=-1)

        # Generate images
        generated_images = generator(conditioned_seed, training=False)
        # Rescale images from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0

        # Display the generated images
        st.subheader("Generated Images:")
        for i in range(num_examples_to_generate):
            # Convert the 28x28x1 image to a displayable format (e.g., PIL image or numpy array)
            img_array = generated_images[i].numpy().squeeze() * 255  # Rescale to 0-255
            img_array = img_array.astype(np.uint8)

            st.image(img_array, caption=f"Generated Digit {digit_to_generate} - Image {i+1}", width=100)

if __name__ == "__main__":
    main()

# Note: To run this Streamlit app in Google Colab, you will need to
# install ngrok or a similar tunneling service to expose the port
# where Streamlit is running.
#
# Example command to run Streamlit and ngrok:
# !streamlit run your_script_name.py &>/dev/null&
# !pip install pyngrok
# from pyngrok import ngrok
# public_url = ngrok.connect(port='8501')
# print(f"Streamlit app is running at: {public_url}")

Training data shape: (36000, 28, 28, 1)
Validation data shape: (12000, 28, 28, 1)
Test data shape: (10000, 28, 28, 1)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 6272)           │       689,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 6272)           │        25,088 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 14, 14, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 28, 28, 1)      │         1,600 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 921,664 (3.52 MB)

 Trainable params: 908,992 (3.47 MB)

...

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

Starting training...
Epoch 1/10
  Generator Loss: 2.5994, Discriminator Loss: 0.3985
Epoch 2/10
  Generator Loss: 1.2453, Discriminator Loss: 0.7466
Epoch 3/10
  Generator Loss: 1.8035, Discriminator Loss: 0.4701
Epoch 4/10
  Generator Loss: 2.3061, Discriminator Loss: 0.3494
Epoch 5/10
  Generator Loss: 2.5245, Discriminator Loss: 0.3283
Epoch 6/10
  Generator Loss: 2.7316, Discriminator Loss: 0.2973
Epoch 7/10
  Generator Loss: 2.8957, Discriminator Loss: 0.2668
Epoch 8/10
  Generator Loss: 3.0521, Discriminator Loss: 0.2481
Epoch 9/10
  Generator Loss: 3.2128, Discriminator Loss: 0.2283
Epoch 10/10


2025-06-22 09:21:37.857 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.950 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-22 09:21:37.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.954 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.955 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 09:21:37.959 Thread 'MainThread': mi

  Generator Loss: 3.3424, Discriminator Loss: 0.2132
Training finished.


In [6]:
# Save the generator model
generator.save('generator_model.h5')
print("Generator model saved successfully.")

Generator model saved successfully.


In [8]:
# prompt: How to deploy the above streamlit model?

# Add this after your Streamlit app code (`if __name__ == "__main__": main()`)

# Note: To run this Streamlit app in Google Colab, you will need to
# install ngrok or a similar tunneling service to expose the port
# where Streamlit is running.
#
# Example command to run Streamlit and ngrok:
# Save your Streamlit code to a Python file
streamlit_code = """
import streamlit as st
import tensorflow as tf
import numpy as np
import random
import os

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Load and preprocess the MNIST dataset (Assuming this is pre-loaded or handled)
# For deployment, you might want to load the saved model instead of retraining
# (x_train_val, y_train_val), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# ... (preprocessing code as in your original script) ...

# Load the trained generator model
try:
    generator = tf.keras.models.load_model('generator_model.h5', compile=False) # compile=False if not compiling for inference
    st.write("Generator model loaded successfully.")
except Exception as e:
    st.error(f"Error loading generator model: {e}")
    st.warning("The model might need to be trained first or the file 'generator_model.h5' is missing.")
    generator = None # Set generator to None if loading fails

noise_dim = 100
num_examples_to_generate = 5

# Streamlit App
def main():
    st.title("Handwritten Digit Generator")
    st.write("Generate grayscale images of handwritten digits using a trained GAN model.")

    if generator is None:
        st.error("Cannot run the app as the model failed to load.")
        return

    digit_to_generate = st.selectbox("Select the digit to generate:", options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    if st.button("Generate Images"):
        st.write(f"Generating {num_examples_to_generate} images of digit {digit_to_generate}...")

        # Create a batch of noise and the specified label
        seed = tf.random.normal([num_examples_to_generate, noise_dim])
        # Create one-hot encoding for the specified digit
        labels = tf.one_hot([digit_to_generate] * num_examples_to_generate, depth=10)
        conditioned_seed = tf.concat([seed, tf.cast(labels, tf.float32)], axis=-1)

        # Generate images
        with st.spinner('Generating images...'):
            generated_images = generator(conditioned_seed, training=False)
            # Rescale images from [-1, 1] to [0, 1]
            generated_images = (generated_images + 1) / 2.0

        # Display the generated images
        st.subheader("Generated Images:")
        for i in range(num_examples_to_generate):
            # Convert the 28x28x1 image to a displayable format (e.g., PIL image or numpy array)
            img_array = generated_images[i].numpy().squeeze() * 255  # Rescale to 0-255
            img_array = img_array.astype(np.uint8)

            st.image(img_array, caption=f"Generated Digit {digit_to_generate} - Image {i+1}", width=100)

if __name__ == "__main__":
    main()
"""

# Save the Streamlit code to a Python file named app.py
with open('app.py', 'w') as f:
    f.write(streamlit_code)

# Install pyngrok if not already installed
!pip install pyngrok

# Start Streamlit in the background
# Redirect output to avoid cluttering the notebook
!streamlit run app.py &>/dev/null&

# Connect ngrok to the Streamlit port (default is 8501)
from pyngrok import ngrok
# Terminate any existing ngrok tunnels to avoid conflicts
ngrok.kill()
public_url = ngrok.connect(port='8501')

print(f"Streamlit app is running at: {public_url}")

# You can now click on the public_url link to access your Streamlit app
# Keep the Colab notebook running to keep the app accessible.
# Closing the Colab notebook will terminate the ngrok tunnel and the Streamlit app.

# To stop the ngrok tunnel and Streamlit app manually later:
# ngrok.kill()
# !pkill streamlit


ERROR:pyngrok.process.ngrok:t=2025-06-22T09:24:37+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-22T09:24:37+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-22T09:24:37+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.